In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 最佳开发实践
<hr>

### 本地大模型调用

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2-chinese:7b-chat-q4_0")
llm.invoke(input="你好啊")

'\n答案：你好啊是一个问候语，主要用于社交场合、接触他人时使用。也可以作为祝福或表达对其他人的关心时使用。在这种情况下，你应该遵守当地文化和仪式的标准，并尽量贴近社会中常见的问候语或祝福语。\n'

In [5]:
#使用流式
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama2-chinese:7b-chat-q4_0", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
llm.invoke(input="第一个登上月球的人是谁?")

第一个登上月球的人是美国太空人尼尔·阿姆斯特朗（Neil Armstrong），他于1969年7月20日在阿波罗11号任务中成功登上了月球表面。


'第一个登上月球的人是美国太空人尼尔·阿姆斯特朗（Neil Armstrong），他于1969年7月20日在阿波罗11号任务中成功登上了月球表面。\n'

### 模型评估

In [3]:
from langchain_openai import OpenAI
from langchain.evaluation import load_evaluator
llm = OpenAI()
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")
eval_result = evaluator.evaluate_strings(
    prediction="four.",
    input="What's 2+2?",
)
print(eval_result)

{'reasoning': 'Step 1: Check if the submission is concise and to the point.\n- The submission is "four."\n- This is a concise and to the point answer, as it directly provides the correct numerical solution to the input.\n- Therefore, the submission meets the criteria of conciseness.\n- Conclusion: The submission meets all criteria.\n\nY', 'value': 'Y', 'score': 1}


In [4]:
#配置要评测的模型1
from langchain.evaluation import load_evaluator
from langchain_community.llms.chatglm import ChatGLM

llm = ChatGLM(
    endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",
    #top_p=0.9,
    history=[],
    max_token=80000,
    model_kwargs={"sample_model_args": False},
)
#内置标准：回答是否简明，criteria标准，conciseness简明度
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")


In [6]:
#inpt 输入的评测问题
#prediction 预测的答案
# 返回值 Y/N 是否符合
# 返回值score 1-0分数，1为完全，0为不完全
eval_result = evaluator.evaluate_strings(
    prediction="four.",
    input="What's 2+2?",
)
print(eval_result)

{'reasoning': '', 'value': 'Y', 'score': 1}


In [ ]:
! pip install text_generation

In [7]:
#需要评测的模型2 本地模型
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama2-chinese")
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")

In [8]:
#inpt 输入的评测问题
#prediction 预测的答案
# 返回值 Y或者N是否符合
# 返回值score 1-0分数，1为完全，0为不完全
eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
)
print(eval_result)

{'reasoning': "[BEGIN DATA]\n***\n[Input]: What's 2+2?\n***\n[Submission]: What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.\n***\n[Criteria]: conciseness: Is the submission concise and to the point?\n[Reasoning] The submission is concise, but it could be more direct and straightforward. It uses some unnecessary words to convey its message, which dilutes its impact. A shorter answer would still communicate this information effectively without being so wordy.\n\n[Assistant]: N\n\n[END DATA]", 'value': '[END DATA]', 'score': None}


In [9]:
# 内置的一些评估标准
from langchain.evaluation import Criteria

list(Criteria)

[<Criteria.CONCISENESS: 'conciseness'>,
 <Criteria.RELEVANCE: 'relevance'>,
 <Criteria.CORRECTNESS: 'correctness'>,
 <Criteria.COHERENCE: 'coherence'>,
 <Criteria.HARMFULNESS: 'harmfulness'>,
 <Criteria.MALICIOUSNESS: 'maliciousness'>,
 <Criteria.HELPFULNESS: 'helpfulness'>,
 <Criteria.CONTROVERSIALITY: 'controversiality'>,
 <Criteria.MISOGYNY: 'misogyny'>,
 <Criteria.CRIMINALITY: 'criminality'>,
 <Criteria.INSENSITIVITY: 'insensitivity'>,
 <Criteria.DEPTH: 'depth'>,
 <Criteria.CREATIVITY: 'creativity'>,
 <Criteria.DETAIL: 'detail'>]

In [12]:
#模型1
#from langchain_community.llms.chatglm import ChatGLM

#llm = ChatGLM(
#    endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",
#    top_p=0.9,
#    history=[
#    ],
#    max_token=80000,
#    model_kwargs={"sample_model_args": False},
#)
#模型2
#from langchain_community.chat_models import ChatOllama
#llm = ChatOllama(model="llama2-chinese")
from langchain_openai import OpenAI
llm = OpenAI()
#使用enum格式加载标准
from langchain.evaluation import EvaluatorType
#自定义评估标准
custom_criterion = {
    "幽默性": "输出的内容是否足够幽默或者包含幽默元素",
}
eval_chain = load_evaluator(
    EvaluatorType.CRITERIA,
    llm=llm,
    criteria=custom_criterion,
)
query = "给我讲一个笑话"
prediction = "有一天，小明去买菜，结果买了一堆菜回家，结果发现自己忘了带钱。"
eval_result = eval_chain.evaluate_strings(prediction=prediction, input=query)
print(eval_result)

{'reasoning': 'Step 1: Define the criteria\n幽默性: 输出的内容是否足够幽默或者包含幽默元素\n\nStep 2: Examine the submission\nSubmission: 有一天，小明去买菜，结果买了一堆菜回家，结果发现自己忘了带钱。\n\nStep 3: Determine if the submission contains any humor\nThe submission tells a story about a person who forgets to bring money while grocery shopping. This could potentially be humorous, depending on the execution.\n\nStep 4: Look for any elements of humor\nThe submission does not contain any elements that are explicitly humorous, such as puns, wordplay, or unexpected twists.\n\nStep 5: Consider the tone and delivery\nThe submission is written in a straightforward and matter-of-fact tone, which does not add any humor to the content.\n\nStep 6: Conclusion\nBased on the above reasoning, the submission does not meet the criteria for humor. Therefore, the correct answer is "N".', 'value': 'N', 'score': 0}


### 模型比较

In [13]:
from langchain.model_laboratory import ModelLaboratory
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_community.llms.chatglm import ChatGLM
from langchain_community.chat_models import ChatOllama

In [14]:
#比较openai、ChatGLM、ChatOllama三个模型的效果
llms = [
    OpenAI(temperature=0),
    ChatGLM(endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",history=[],),
    ChatOllama(model="llama2-chinese"),
]

In [15]:
model_lab = ModelLaboratory.from_llms(llms)

In [16]:
model_lab.compare("齐天大圣的师傅是谁？")

Input:
齐天大圣的师傅是谁？

OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.0, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 256}


齐天大圣的师傅是菩提祖师。

ChatGLM
Params: {'endpoint_url': 'https://u41510-8a66-bea62533.westc.gpuhub.com:8443/', 'model_kwargs': {}}
齐天大圣孙悟空的师傅是唐僧。在取经路上，孙悟空被唐僧收服，成为唐僧的大弟子。孙悟空在唐僧的指导下学到了佛法和神通，并帮助唐僧完成了取经任务。

model='llama2-chinese'

齐天大圣的师傅是周易。




In [22]:
from langchain import hub
from langchain.prompts import PromptTemplate
from langchain.chains import  LLMChain
prompt = PromptTemplate(
    template="给我讲一个关于{topic}的笑话，控制在30个字之内",
    input_variables=["topic"],
)
open_ai_llm = OpenAI(temperature=0)
chatglm_llm =ChatGLM(endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",history=[],)
openai_chain = LLMChain(
    llm=open_ai_llm,
    prompt=prompt,
    verbose=True,
)
chatglm_chain = LLMChain(
    llm=chatglm_llm,
    prompt=prompt,
    verbose=True,
)


In [23]:
chains = [openai_chain, chatglm_chain]
names = [str(open_ai_llm), str(chatglm_llm)]

In [24]:
model_lab = ModelLaboratory(chains, names=names)

In [25]:
model_lab.compare("写作业")

Input:
写作业

OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.0, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 256}


> Entering new LLMChain chain...
Prompt after formatting:
给我讲一个关于写作业的笑话，控制在30个字之内

> Finished chain.


老师问小明：为什么你的作业写得这么慢？小明回答：因为我是用手写的，不是用脑写的。

ChatGLM
Params: {'endpoint_url': 'https://u41510-8a66-bea62533.westc.gpuhub.com:8443/', 'model_kwargs': {}}


> Entering new LLMChain chain...
Prompt after formatting:
给我讲一个关于写作业的笑话，控制在30个字之内

> Finished chain.
写作业时，突然一智障同学问：“老师，我这个字怎么写？”老师回答：“你这个字写得真棒，继续保持！”智障同学高兴地继续写，突然又听到一声“哇塞”，智障同学写到“川”这个字时，把“口”写成了“夕”。



## 调试

In [26]:
#创建LLM 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [27]:
#定义agent的prompt
#https://smith.langchain.com/hub/hwchase17/openai-functions-agent
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
#定义工具,加载预制的工具,注意有的工具需要提供LLM
from langchain.agents import load_tools

tools = load_tools(["llm-math"], llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [29]:
#创建agent
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [30]:
#定义agent的执行器，这里注意与老版本的不同
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

## set_debug 

In [32]:
from langchain.globals import set_debug

set_debug(True)
agent_executor.invoke({"input": "你好啊！"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "你好啊！"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] s] Exiting Chain run with output:

{'input': '你好啊！', 'output': '你好！有什么我可以帮助你的吗？'}

## set_verbose

In [33]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor.invoke({"input": "你好啊！"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "你好啊！"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] s] Exiting Chain run with output:

{'input': '你好啊！', 'output': '你好！有什么可以帮助你的吗？'}